In [1]:
import os
import re
import sys
import glob
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm
from uuid import uuid4
from functools import reduce
from multiprocessing import Pool
from sentence_transformers import SentenceTransformer
import faiss
import nltk
nltk.download('punkt_tab')
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /home/hlopin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hlopin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Извлечение текстов википедии из архива. Из архива достаём различные страницы из википедии, которые потом будем добавлять в базы данных

In [2]:
#!pip install wikiextractor

In [3]:
import wikiextractor

Скачиваем архив

In [4]:
#!wget http://dumps.wikimedia.org/ruwiki/latest/ruwiki-latest-pages-articles.xml.bz2

Разархивируем в ./wiki папку. Для начала лучше разархивировать только несколько папок с документами по википедии (AA,AB)

In [5]:
# !python3 -m wikiextractor.WikiExtractor -o ./data/wiki/ --no-templates --processes 8 ./ruwiki-latest-pages-articles.xml.bz2

In [6]:
# !python3 -m wikiextractor.extractPage -o ./data/wiki/ --no-templates --processes 8 ./ruwiki-latest-pages-articles.xml.bz2

## Функционал для парсинг текстов википедии

Вспомогательные функции

In [7]:
def _remove_non_printed_chars(string):
    reg = re.compile('[^a-zA-Zа-яА-ЯёЁ]')
    # reg = re.compile('[^\wёЁ]')
    return reg.sub(' ', string)

def _remove_stop_words(string,sw=[]):
    return ' '.join([word if word not in sw else '' \
                     for word in string.strip().split(' ')])

def _trim_string(string):
    # remove extra spaces, remove trailing spaces, lower the case 
    return re.sub('\s+',' ',string).strip().lower()
    
def clean_string(string,
                 stop_words_list,
                 min_len=2,
                 max_len=30):

    string = _remove_non_printed_chars(string)
    string = _remove_stop_words(string, stop_words_list)
    string = _trim_string(string)
    # also remove short words, most likely containing addresses / crap / left-overs / etc remaining after removal
    # gensim mostly does the same as above, it is used here for simplicity
    string = ' '.join(gensim.utils.simple_preprocess(string,
                                                     min_len=min_len,
                                                     max_len=max_len))
    return string
    
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
    
def remove_special_chars(text,char_list):
    for char in char_list:
        text=text.replace(char,'')
    return text.replace(u'\xa0', u' ')

def splitkeepsep(s, sep):
    cleaned = []
    s = re.split("(%s)" % re.escape(sep), s)
    for _ in s:
        if _!='' and _!=sep:
            cleaned.append(sep+_)
    return cleaned

def extract_url(text):
    pattern = 'http([^"]+)'
    match = re.search(pattern, text)
    if match:
        url = match.group(0)
        return url
    else:
        return ""

def create_vector(text):
    return model.encode(text, normalize_embeddings=True)

Функция для извлечения списка оглавления из страницы википедии

In [8]:
import requests
from bs4 import BeautifulSoup

# URL статьи Википедии
url = 'https://ru.wikipedia.org/wiki?curid=9'

def getHeadings(url):
    # Получаем содержимое страницы
    response = requests.get(url)
    
    # Парсим HTML-код с помощью BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Находим элемент с оглавлением (обычно он находится внутри элемента с классом mw-parser-output)
    toc = soup.find('div', id='toc')
    if toc is None:
        return None
    
    # Извлекаем все элементы списка (<li>) из оглавления
    items = toc.find_all('li')
    
    # Формируем список заголовков
    headings = []
    for item in items:
        link = item.find('a')  # находим ссылку внутри каждого пункта списка
        if link is not None:
            heading_text = link.text.strip()  # получаем текст ссылки
            cleaned_heading = heading_text.split(maxsplit=1)[-1].strip()  # убираем номер и точку
            if cleaned_heading + '.' not in headings:
                headings.append(cleaned_heading + '.')  # добавляем очищенное название в список
    
    # Выводим результат
    return headings

Основная функция для обработки скаченных текстов википедии

In [9]:
from nltk.stem.snowball import SnowballStemmer

def stem(doc):
   stemmer = SnowballStemmer("russian")
   return [stemmer.stem(word) for word in doc.split()]

def process_wiki_files(wiki_file):
    chars = ['\n\n']
    global sw

    with open(wiki_file, encoding='utf-8') as f:
        content = f.read()

    articles = splitkeepsep(content,'<doc id=')
    df_texts = pd.DataFrame(columns=['article_uuid','url', 'title', 'article','proc_article','proc_len'])
    emds = []

    for article in articles:
        if len(article) < 500:
            continue

        uuid_text = uuid4()
        
        articleParts = article.split('\n')
        url = extract_url(article)
        headings = getHeadings(url)
        if headings is None:
            continue
        title = articleParts[1]

        article = remove_html_tags(article)
        article = remove_special_chars(article, chars)
        clearArticleParts = article.split('\n')
        
        startIndex = 1
        currHeading = ''
        
        for endIndex in range(startIndex, len(clearArticleParts)):
            if len(clearArticleParts[endIndex]) < 100 and clearArticleParts[endIndex] in headings: 
                if endIndex - startIndex == 1:
                    startIndex = endIndex
                    currHeading = clearArticleParts[endIndex]
                    continue
            
                onePart = title + '. ' + currHeading + ' ' + ' '.join(clearArticleParts[startIndex+1:endIndex])
            
                proc_onePart = clean_string(onePart, sw_ru)
                proc_len = len(proc_onePart.split(' '))

                stemmed_onePart = ' '.join(stem(proc_onePart))
            
                temp_df_texts = pd.DataFrame(
                    {'article_uuid': [uuid_text],
                     'url': url + "#" + currHeading[:-1].replace(' ', '_') if len(currHeading) > 0 else url,
                     'title': title + '. ' + currHeading if len(currHeading) > 0 else title,
                     'article': onePart,
                     'proc_article':proc_onePart,
                     'proc_len':proc_len,
                     'stem_article':stemmed_onePart
                    })
                df_texts = pd.concat([df_texts, temp_df_texts], ignore_index=True)
            
                emb = create_vector(proc_onePart)
                emds.append(emb)
            
                startIndex = endIndex
                currHeading = clearArticleParts[endIndex]
    
    return df_texts, np.array(emds)

sw_en = set(stopwords.words('english'))
sw_ru = set(stopwords.words('russian'))
sw = list(sw_ru.union(sw_en))  

Получаем модель для эмбеддингов

In [10]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

/home/hlopin/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


## Функции для работы с базами данных

### Векторная БД

#### Сохранение векторов в векторную БД с обратным индексом (используем Faiss)

In [11]:
import os
import faiss
from os.path import exists

def saveEmbdsToVectorDB(embds, path, indexType = faiss.IndexFlatL2):
    if not exists(path):
        index = indexType(embds.shape[1])
        index = faiss.IndexIDMap(index)
        index.add_with_ids(embds, np.arange(0, embds.shape[0]))
        faiss.write_index(index, path)
    else:
        index = faiss.read_index(path)
        index.add_with_ids(embds, np.arange(index.ntotal, index.ntotal + embds.shape[0]))
        faiss.write_index(index, path)

#### Получаем векторную БД

In [12]:
def getVectorDB(path):
    return faiss.read_index(path)

###  Текстовые БД

#### Сохранение в SQLlite БД метаданных (название, url и т.д.)

In [13]:
def addMetadataToDB(pathDB, cursor, conn, metadataDf):
    metadataDf.to_sql(name='documents', con=conn, if_exists='append', index=True)
    conn.commit()

#### Поиск в текстовой БД по запросу

Получаем тексты по индексам

In [14]:
def get_rows_from_csv(filename, indices):
    df = pd.read_csv(
        filename,
        header=None,
        skiprows=lambda x: x not in indices
    )
    
    return df

In [15]:
def textSearch_with_bm25_ranking(query, pathDB):
    index = open_dir(pathDB)
    with index.searcher() as searcher:
        query_parser = QueryParser("content", index.schema)
        parsed_query = query_parser.parse(query)
        print("Получился запрос вида: ", parsed_query)
        results = searcher.search(parsed_query)
        return np.array([(result['id'], result.score) for result in results])

## Заполнение баз данных. (Объединяем всё вместе, парсим документы и сохраняем их в базы данных)

In [16]:
wikiFilesRootPath = "data/wiki"
vectorDBPath = 'data/data_bases/vectorDB.index'
metadataDBPath = "data/data_bases/documentsMetadataDB.db"
textsCsvPath = "data/data_bases/texts.csv"

In [17]:
import sqlite3
import os

import threading

# Инициализируем блокировку
lock = threading.Lock()

currentDbSize = 0
if os.path.exists(vectorDBPath):
    vectDb = getVectorDB(vectorDBPath)
    currentDbSize = vectDb.ntotal

def process_file(file_path):
    global currentDbSize
    print("Обрабатываю: ", file_path)
    
    # сохраняем в векторную БД
    df_texts, embds = process_wiki_files(file_path)
    saveEmbdsToVectorDB(embds, vectorDBPath)
    
    # сохраняем тексты документов в текстовую БД
    df_texts.to_csv(textsCsvPath, mode='a', header=False)

    with lock:
        conn = sqlite3.connect(metadataDBPath)
        cursor = conn.cursor()
        # сохраняем метаданные документов в SQLlite БД
        new_index = range(currentDbSize, currentDbSize + len(df_texts))
        currentDbSize += len(df_texts)
        df_texts.index = new_index
        addMetadataToDB(metadataDBPath, cursor, conn, df_texts[['url', 'title', 'article', 'proc_article', 'stem_article']])
        conn.close()

Параллельно обрабатыватексты тексты и сохраяем результаты в базы данных

In [18]:
# from concurrent.futures import ThreadPoolExecutor

# files_to_process = []
# for dirpath, dirnames, filenames in os.walk(wikiFilesRootPath):
#     for filename in filenames:
#         file_path = os.path.join(dirpath, filename)
#         files_to_process.append(file_path)

#  # Используем ThreadPoolExecutor для параллельной обработки файлов
# with ThreadPoolExecutor(max_workers=8) as executor:  # Количество рабочих потоков можно настроить
#     futures = {executor.submit(process_file, file_path): file_path for file_path in files_to_process}
    
#     # Ждем завершения всех задач
#     for future in concurrent.futures.as_completed(futures):
#         file_path = futures[future]
#         try:
#             data = future.result()
#         except Exception as exc:
#             print(f'Ошибка при обработке файла {file_path}: {exc}')

#### Проверка на адекватность векторной БД

In [39]:
index = getVectorDB(vectorDBPath)

texts = get_rows_from_csv(textsCsvPath, range(30))[5][:30]
first_30_vectors = model.encode(texts, normalize_embeddings=True)
    
# Находим соседей для первых 30 векторов в БД
D, I = index.search(np.array(first_30_vectors), 12)
# Должны быть те же индексы от 0 до 29. То есть те же векторы, для которых искали соседей. (там где не совпадает, это потомучто вместо текста Nan, снизу видно)
print("Индексы похожих документов:", I[0:30, 0])


RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char*) at /project/faiss/faiss/impl/io.cpp:67: Error: 'f' failed: could not open data/data_bases/vectorDB.index for reading: No such file or directory

In [21]:
texts

0     аничков мост первый деревянный мост построен г...
1     аничков мост расположение расположен оси невск...
2     аничков мост название название моста произошло...
3     аничков мост деревянный мост до годов фонтанка...
4     аничков мост каменный мост образца годов серед...
5     аничков мост современный мост году принято реш...
6     аничков мост капитальный ремонт годов эксплуат...
7     аничков мост капитальный ремонт годов году свя...
8     аничков мост скульптурные группы укрощение кон...
9     аутсорсинг аутсорсинг позволяет повысить эффек...
10    аутсорсинг производственный аутсорсинг при про...
11    аутсорсинг ит аутсорсинг ito предполагает деле...
12    аутсорсинг аутсорсинг управления знаниями kpo ...
13    аутсорсинг аутсорсинг силу закона ряде случаев...
14    аутсорсинг наиболее распространённые формы аут...
15    аутсорсинг аутсорсинг сфере ит ит аутсорсинг ч...
16    аутсорсинг обслуживание инфокоммуникационных с...
17    аутсорсинг аутсорсинг цод во многих отрасл

## Процесс поиска

Здесь мы находим kDocuments релевантных документов с помощью векторного поиска и далее ранжируем

In [22]:
# !pip install pymorphy2 rank_bm25

In [23]:
import nltk
from nltk.stem.snowball import SnowballStemmer

# Загрузка необходимых ресурсов
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/hlopin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Функционал для ранжирования

In [24]:
from abc import ABC, abstractmethod
from rank_bm25 import BM25Okapi
from sentence_transformers import CrossEncoder
from abc import ABC, abstractmethod
# from colbert import ColBERT
# import colbert

class DocsRanker(ABC):
    @abstractmethod
    def rankDocuments(self, query, docs):
        pass


class Bm25Ranker(DocsRanker):
    # preprocess_func: переобразует запрос и документ в список слов
    def __init__(self, preprocess_func = None) -> None:
        self.preprocess_func = preprocess_func

    def rankDocuments(self, query, docs):
        if self.preprocess_func is None:
            self.preprocess_func = lambda doc: doc.split()
        tokenized_corpus = [self.preprocess_func(doc) for doc in docs]
        bm25 = BM25Okapi(tokenized_corpus)
        tokenized_query = self.preprocess_func(query)
        return bm25.get_scores(tokenized_query)

def lemmatize(doc):
    morph = pymorphy2.MorphAnalyzer()
    return [morph.parse(word)[0].normal_form for word in doc.split()]

def stem(doc):
   stemmer = SnowballStemmer("russian")
   words = nltk.word_tokenize(doc, language="russian")
   return [stemmer.stem(word) for word in words]


class CrossEncoderRanker(DocsRanker):
    def __init__(self) -> None:
        # self.reranker_model = CrossEncoder('DiTy/cross-encoder-russian-msmarco', max_length=512, device='cuda')
        self.reranker_model = CrossEncoder('DiTy/cross-encoder-russian-msmarco', max_length=512, device='cpu')

    def rankDocuments(self, query, docs):
        return np.array([self.reranker_model.predict([[query, doc]])[0] for doc in docs])

# class ColBERTRanker(DocsRanker):
#     def __init__(self) -> None:
#         self.model = ColBERT(
#             model_name="bert-base-uncased",  # или любая другая модель на основе BERT
#             similarity_function='dot_product',  # или другой метод, если это требуется
#             # Дополнительные параметры могут быть добавлены по мере необходимости
#         )

#     def rankDocuments(self, query, docs):
#         scores = self.model.rank(query, docs)  # Это пример, конкретная реализация может отличаться
#         sorted_docs = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
#         return sorted_docs

Функционал для полного сценария поиска

In [25]:
from rank_bm25 import BM25Okapi
import pymorphy2
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast

tokenizer = T5TokenizerFast.from_pretrained('UrukHan/t5-russian-summarization')
model_summarizer = AutoModelForSeq2SeqLM.from_pretrained('UrukHan/t5-russian-summarization')


def summarizeText(docs, query = None):
    global model_summarizer
    global tokenizer

    max_input = 1512
    task_prefix = "" #"Find answer on question: "
    input_seq = ""
    if query != None:
        input_seq = " " + query + "\n" + "В тексте: "
    input_seq += "\n".join(docs)
    print(input_seq)
    input_seq = [input_seq]
    encoded = tokenizer(
      [task_prefix + sequence for sequence in input_seq],
      padding="longest",
      max_length=max_input,
      truncation=True,
      return_tensors="pt",
    )
    
    predicts = model_summarizer.generate(encoded['input_ids'])    # # Прогнозирование
    
    return tokenizer.batch_decode(predicts, skip_special_tokens=True)  # Декодируем данные

def findVectorsIndexes(query, encoder, kDocuments):
    index = getVectorDB(vectorDBPath)
    queryEmbd = encoder.encode(query, normalize_embeddings=True)
    D, I = index.search(np.array([queryEmbd]), kDocuments)
    return I[0]

def retrieveDocsAndUrls(indexes):
    urlsAndDocs = get_rows_from_csv(textsCsvPath, indexes)[[2, 4, 5]]
    urlsAndDocs = urlsAndDocs.fillna('stub')
    return urlsAndDocs[2], urlsAndDocs[4]

def rankDocuments(query, indexes, ranker):
    urls, docs = retrieveDocsAndUrls(indexes)
    doc_scores = ranker.rankDocuments(query, docs)
    sorted_idx = np.argsort(doc_scores)
    return list(docs.iloc[sorted_idx[::-1]]), list(urls.iloc[sorted_idx[::-1]]), doc_scores[sorted_idx[::-1]]

def getSortedDocumentsWithUrls(query, encoder, kDocuments, ranker):
    indexes = findVectorsIndexes(query, encoder, kDocuments)
    return rankDocuments(query, indexes, ranker)

def summarize_documents(query, docs, max_length, min_length):
    global summarizer

    full_text = [query]
    for doc in docs:
        full_text.append(doc)

    return summarizer('\n'.join(full_text), max_length=max_length, min_length=min_length, do_sample=False)
    

In [80]:
query = "Самое крупное государство"
kDocuments = 50
docs, urls, bm25_scores = getSortedDocumentsWithUrls(query, model, kDocuments, Bm25Ranker())
df = pd.DataFrame({'relevant docs': docs, 'bm25 scores': bm25_scores})
print(df)

                                        relevant docs  bm25 scores
0   Земноводные. Размеры. Размеры представителей г...     2.819909
1   Чад.  Столица — город Нджамена. Государственны...     2.094205
2   Румыния.  Площадь территории — 238 391 км², на...     1.911755
3   Содружество наций. Членство. Общая численность...     1.262955
4   Союз Советских Социалистических Республик.  СС...     0.988344
5   Географические координаты. Географическая сист...     0.000000
6   Китай.  С востока страна омывается водами запа...     0.000000
7   Маркс, Карл. Лондонская эмиграция. Карл Маркс ...     0.000000
8   Хакасия. Рельеф. Преобладающий рельеф местност...     0.000000
9   Эль-Барадеи, Мохаммед. Участие в президентских...     0.000000
10  Шанхайская организация сотрудничества.  Общая ...     0.000000
11  Союз Советских Социалистических Республик. Гео...     0.000000
12  Дальневосточный федеральный округ. Национальны...     0.000000
13  Соединённые Штаты Америки. Авиатранспорт. Соед...     0.00

In [29]:
query = "Самое крупное государство"
kDocuments = 50
docs, urls, bm25_scores = getSortedDocumentsWithUrls(query, model, kDocuments, Bm25Ranker(lemmatize))
df = pd.DataFrame({'relevant docs': docs, 'bm25 scores': bm25_scores})
print(df)

                                        relevant docs  bm25 scores
0   моравско силезский край города наиболее крупны...     4.333728
1   новософт история создания novosoft ооо новософ...     3.790155
2   антакальнис общая характеристика сооружённые в...     2.873437
3   обыкновенная пиранья это один из самых известн...     2.540833
4   соболь антропонимика в московской руси наимено...     2.003871
5   кокандское ханство экономика и нумизматика в с...     1.908181
6   ветер значение в мифологии и культуре во многи...     1.337799
7   final fantasy vii технологическая часть непоср...     1.184804
8   петров евгений петрович рассказы и фельетоны и...     1.148489
9   алания в эпоху татар в году объединённые силы ...     0.725435
10  рябов яков петрович карьера в цк кпсс член цк ...     0.000000
11  лафферти джеймс карьера большую часть своего д...     0.000000
12  багратионовский район история современный багр...     0.000000
13  серотонин метаболизм анаболизм и катаболизм се...     0.00

In [30]:
query = "Самое крупное государство"
kDocuments = 50
docs, urls, bm25_scores = getSortedDocumentsWithUrls(query, model, kDocuments, Bm25Ranker(stem))
df = pd.DataFrame({'relevant docs': docs, 'bm25 scores': bm25_scores})
print(df)

                                        relevant docs  bm25 scores
0   моравско силезский край города наиболее крупны...     4.333728
1   новософт история создания novosoft ооо новософ...     3.790155
2   антакальнис общая характеристика сооружённые в...     2.696624
3   обыкновенная пиранья это один из самых известн...     2.131141
4   modbus запись нескольких значений команда сост...     2.038555
5   соболь антропонимика в московской руси наимено...     1.680760
6   кокандское ханство экономика и нумизматика в с...     1.600500
7   петров евгений петрович рассказы и фельетоны и...     1.264488
8   ветер значение в мифологии и культуре во многи...     1.122087
9   final fantasy vii технологическая часть непоср...     0.993762
10  солод получение процесс получения солода делит...     0.715100
11  алания в эпоху татар в году объединённые силы ...     0.608463
12  рябов яков петрович карьера в цк кпсс член цк ...     0.000000
13  обратная транскриптаза значение для вирусов об...     0.00

In [26]:
query = "Самое крупное государство"
kDocuments = 50

indexes = findVectorsIndexes(query, encoder, kDocuments)
docs, urls, scores = rankDocuments(query, indexes, ranker)


# docs, urls, scores = getSortedDocumentsWithUrls(query, model, kDocuments, CrossEncoderRanker())
df = pd.DataFrame({'relevant docs': docs, 'scores': scores})
print(df)

                                        relevant docs    scores
0   География Калифорнии.  Калифорния — крупнейший...  0.822690
1   Барсум. Гелиум. Двойной город (Малый Гелиум от...  0.613793
2   Страны Африки, Карибского бассейна и Тихого ок...  0.478108
3   Торент. География. Королевство Торент — одно и...  0.033331
4   Эксума. Большой Эксума. Большой Эксума являетс...  0.024914
5   Маяк мыса Хаттерас.  Является самым высоким ма...  0.009683
6   Астровитянка. Правление. Существующие на Земле...  0.006624
7   Северо-Восток США. Состав. Регион состоит из д...  0.004310
8    Жизор (Эр).  Население (2018) — 11 674 человека.  0.002519
9   Такна (регион).  Административный центр регион...  0.002313
10  Пенжинская ПЭС.  В зависимости от выбранного п...  0.002121
11      Конкарно.  Население (2019) — 19 816 человек.  0.001664
12      Верх-Тюш.  Население — 265 человек(2022 год).  0.001340
13  Нампула (город). Население. С численностью нас...  0.001306
14  Верхошижемье.  Население 4108 жителе

In [58]:
# query = "Самое крупное государство"
# kDocuments = 50
# docs, urls, scores = getSortedDocumentsWithUrls(query, model, kDocuments, ColBERTRanker())
# df = pd.DataFrame({'relevant docs': docs, 'scores': scores})
# print(df)

Поиск с моделью-суммаризатором

In [94]:
query = "Какой самый большой город на земле?"
kDocuments = 100
docs, urls, scores = getSortedDocumentsWithUrls(query, model, kDocuments, CrossEncoderRanker())
df = pd.DataFrame({'relevant docs': docs, 'scores': scores})
print(df)

                                        relevant docs    scores
0   Германия. Города. Самыми крупными городами Гер...  0.253303
1   Китай. Крупнейшие города. Начиная с 2000 г., к...  0.227778
2   Токио.  Помимо столицы, Токио также является о...  0.145966
3   Канны. Соборы и церкви. Самым большим парком г...  0.120581
4   Помпеи. Дом Юлии Феликс. Занимает одну из самы...  0.089081
..                                                ...       ...
95  Вожегодский район. Урбанизация. Городское насе...  0.001030
96  Саппоро. Население. Население города составляе...  0.001028
97  Киото. Население. Население города составляет ...  0.001028
98  Ковернинский район. География. Район занимает ...  0.001028
99  Судак (город). Национальный состав. По данным ...  0.001025

[100 rows x 2 columns]


In [98]:
docs[:5]

['Германия. Города. Самыми крупными городами Германии являются Берлин, Гамбург, Мюнхен и Кёльн. Следующим по значимости является пятый по численности населения город Германии и финансовая метрополия Франкфурт-на-Майне, самый крупный аэропорт Германии. Это третий по размерам аэропорт Европы и первый по объёмам прибыли от грузовых авиаперевозок. Рурский бассейн — регион с самой высокой плотностью населения.',
 'Китай. Крупнейшие города. Начиная с 2000 г., китайские города увеличивались в среднем на 10 % в год. В настоящее время в КНР есть более 100 городов с населением свыше одного миллиона жителей.',
 'Токио.  Помимо столицы, Токио также является одной из сорока семи префектур страны. По состоянию на 1 августа 2021 года население Токио составляло , что делает Токио префектурой с самым большим населением в Японии. Большой Токио является городской территорией, городской агломерацией и метрополитенским ареалом с самым большим населением в мире, с населением по состоянию на 2016 год в , явл

In [109]:
summary = summarizeText(docs[:5], query)
print(f"Суммаризация текста:\n{summary}\n\n")

 Какой самый большой город на земле?
В тексте: Германия. Города. Самыми крупными городами Германии являются Берлин, Гамбург, Мюнхен и Кёльн. Следующим по значимости является пятый по численности населения город Германии и финансовая метрополия Франкфурт-на-Майне, самый крупный аэропорт Германии. Это третий по размерам аэропорт Европы и первый по объёмам прибыли от грузовых авиаперевозок. Рурский бассейн — регион с самой высокой плотностью населения.
Китай. Крупнейшие города. Начиная с 2000 г., китайские города увеличивались в среднем на 10 % в год. В настоящее время в КНР есть более 100 городов с населением свыше одного миллиона жителей.
Токио.  Помимо столицы, Токио также является одной из сорока семи префектур страны. По состоянию на 1 августа 2021 года население Токио составляло , что делает Токио префектурой с самым большим населением в Японии. Большой Токио является городской территорией, городской агломерацией и метрополитенским ареалом с самым большим населением в мире, с населе

## Пример использования текстовой SQL БД

In [119]:
def get_row_by_positions(cursor, table_name, indexes):    
    indexes_str = ', '.join(str(ind) for ind in indexes)
    query = f"SELECT url, proc_article, article FROM {table_name} WHERE \"index\" IN ({indexes_str})"
    cursor.execute(query)
    return pd.DataFrame(cursor.fetchall())

In [120]:
conn = sqlite3.connect('data/data_bases/documentsMetadataDB.db')
cursor = conn.cursor()

In [121]:
rows = get_row_by_positions(cursor, 'documents', [0, 1,2,3,4])

In [122]:
rows

,0,1,2
0,https://ru.wikipedia.org/wiki?curid=59357,аничков мост первый деревянный мост построен г...,Аничков мост. Первый деревянный мост построен...
1,https://ru.wikipedia.org/wiki?curid=59357#Расп...,аничков мост расположение расположен оси невск...,Аничков мост. Расположение. Расположен по оси ...
2,https://ru.wikipedia.org/wiki?curid=59357#Назв...,аничков мост название название моста произошло...,Аничков мост. Название. Название моста произош...
3,https://ru.wikipedia.org/wiki?curid=59357#Дере...,аничков мост деревянный мост до годов фонтанка...,Аничков мост. Деревянный мост. До 1712—1714 го...
4,https://ru.wikipedia.org/wiki?curid=59357#Каме...,аничков мост каменный мост образца годов серед...,Аничков мост. Каменный мост образца 1780-х год...


In [ ]:
conn.close()

Проверям совпадают ли порядки строк в разных текстовых БД

In [32]:
texts = pd.read_csv('data/data_bases/texts.csv', header=None,)

In [33]:
texts.head(5)

,0,1,2,3,4,5,6
0,0,53eb3616-89cd-439d-868f-71d8681342b8,https://ru.wikipedia.org/wiki?curid=1895123#Ге...,Пожешко-Славонска. География.,Пожешко-Славонска. География. Площадь жупании ...,пожешко славонска география площадь жупании 18...,96
1,1,53eb3616-89cd-439d-868f-71d8681342b8,https://ru.wikipedia.org/wiki?curid=1895123#На...,Пожешко-Славонска. Население.,Пожешко-Славонска. Население. В соответствии с...,пожешко славонска население в соответствии с д...,32
2,2,e1d0c73e-000f-49a5-b9e4-dddba9a68acd,https://ru.wikipedia.org/wiki?curid=1895137,Пажитник голубой,Пажитник голубой. Родина вида — Средиземномор...,пажитник голубой родина вида средиземноморье н...,22
3,3,e1d0c73e-000f-49a5-b9e4-dddba9a68acd,https://ru.wikipedia.org/wiki?curid=1895137#Бо...,Пажитник голубой. Ботаническое описание.,Пажитник голубой. Ботаническое описание. Однол...,пажитник голубой ботаническое описание однолет...,100
4,4,e1d0c73e-000f-49a5-b9e4-dddba9a68acd,https://ru.wikipedia.org/wiki?curid=1895137#Хи...,Пажитник голубой. Химический состав.,Пажитник голубой. Химический состав. В семенах...,пажитник голубой химический состав в семенах п...,66


In [22]:
conn = sqlite3.connect('data/data_bases/sharded/documentsMetadataDB_1.db')
cursor = conn.cursor()
query = f"SELECT * FROM documents"
cursor.execute(query)
sqlDf = pd.DataFrame(cursor.fetchall())

In [24]:
index = getVectorDB('data/data_bases/sharded/vectorDB_1.index')

texts = sqlDf[4][:30]
first_30_vectors = model.encode(texts, normalize_embeddings=True)
    
# Находим соседей для первых 30 векторов в БД
D, I = index.search(np.array(first_30_vectors), 12)
# Должны быть те же индексы от 0 до 29. То есть те же векторы, для которых искали соседей. (там где не совпадает, это потомучто вместо текста Nan, снизу видно)
print("Индексы похожих документов:", I[0:30, 0])


Индексы похожих документов: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]


In [23]:
sqlDf[4][:30]

,0,1,2,3,4
0,0,https://ru.wikipedia.org/wiki?curid=59357,Аничков мост,Аничков мост. Первый деревянный мост построен...,аничков мост первый деревянный мост построен г...
1,1,https://ru.wikipedia.org/wiki?curid=59357#Расп...,Аничков мост. Расположение.,Аничков мост. Расположение. Расположен по оси ...,аничков мост расположение расположен оси невск...
2,2,https://ru.wikipedia.org/wiki?curid=59357#Назв...,Аничков мост. Название.,Аничков мост. Название. Название моста произош...,аничков мост название название моста произошло...
3,3,https://ru.wikipedia.org/wiki?curid=59357#Дере...,Аничков мост. Деревянный мост.,Аничков мост. Деревянный мост. До 1712—1714 го...,аничков мост деревянный мост до годов фонтанка...
4,4,https://ru.wikipedia.org/wiki?curid=59357#Каме...,Аничков мост. Каменный мост образца 1780-х годов.,Аничков мост. Каменный мост образца 1780-х год...,аничков мост каменный мост образца годов серед...
...,...,...,...,...,...
75355,75355,https://ru.wikipedia.org/wiki?curid=25505#Личн...,"Багратион, Пётр Иванович. Личная жизнь Баграти...","Багратион, Пётр Иванович. Личная жизнь Баграти...",багратион пётр иванович личная жизнь багратион...
75356,75356,https://ru.wikipedia.org/wiki?curid=25505#Награды,"Багратион, Пётр Иванович. Награды.","Багратион, Пётр Иванович. Награды. Российские ...",багратион пётр иванович награды российские ино...
75357,75357,https://ru.wikipedia.org/wiki?curid=25505#Памя...,"Багратион, Пётр Иванович. Память о Багратионе.","Багратион, Пётр Иванович. Память о Багратионе....",багратион пётр иванович память багратионе июля...
75358,75358,https://ru.wikipedia.org/wiki?curid=25505#В_ар...,"Багратион, Пётр Иванович. В архитектуре.","Багратион, Пётр Иванович. В архитектуре. В 199...",багратион пётр иванович архитектуре москве отк...


# Итог (устаревшее)

### Что имеем.

У нас есть архив(ы) со страницами с википедии, из которых мы составляем:
1) Векторную БД (Faiss)
2) Тестовую БД, работающую на обратном индексе (библиотека Whoosh, которая работает по принципу Lucene)
3) csv файл, в котором хранятся все спаршенные страницы

Пока что всё таким образом (храним лишнюю csv, которая к тому же содержит полные тексты документов), потому что только начинаем разрабать и тестировать поисковик. Поэтому нужно от этого будет избавиться и сделать всё эффективнее.



**Как сейчас ведётся поиск**.
1) Получаем текстовый запрос
2) Переводим его в вектор
3) Находим k документов из векторной БД, которые наибольшим образом похожи на запрос

### Что нужно сделать дальше

Нужно как-то отранжировать полученные k документов из векторной БД. 

Моё изначальное предложение было - отранжировать текстово с использованием алгоритма BM25. Можно так и сделать. 

Или же предлагайте новые варианты.